<a href="https://colab.research.google.com/github/Kumarbegnier/Computer-Vision-Projects/blob/main/Think_Act_Rise_Foundation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Install Required Libraries

In [35]:
!pip install requests beautifulsoup4


 Import Modules and Define Globals

In [36]:
import requests
from bs4 import BeautifulSoup
from datetime import datetime, timedelta
import json
import os

BASE_URL = "https://services.ecourts.gov.in/ecourtindia_v6/"


Define User Input Function

In [37]:
def get_user_input():
    cnr = input("Enter CNR number (leave blank if unknown): ").strip() or None
    case_type = input("Enter Case Type (if no CNR): ").strip() or None
    case_no = input("Enter Case Number (if no CNR): ").strip() or None
    case_year = input("Enter Case Year (if no CNR): ").strip() or None
    check_today = input("Check listings for today? (y/n): ").lower() == 'y'
    check_tomorrow = input("Check listings for tomorrow? (y/n): ").lower() == 'y'
    download_pdf = input("Download PDF if available? (y/n): ").lower() == 'y'
    download_cause_list = input("Download full cause list for today? (y/n): ").lower() == 'y'
    api_key = input("Enter your E-Courts API key (leave blank to skip API): ").strip() or None
    return (cnr, case_type, case_no, case_year, check_today, check_tomorrow,
            download_pdf, download_cause_list, api_key)


Define Function to Fetch Case Details from API

In [38]:
def fetch_case_details_api(cnr, api_key):
    if not cnr or not api_key:
        print("CNR or API key missing; skipping API call.")
        return None
    url = f"https://court-api.kleopatra.io/search/cnr/{cnr}"
    headers = {'Authorization': f'Bearer {api_key}'}
    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        return response.json()
    else:
        print(f"API error {response.status_code}: {response.text}")
        return None


 Define Scraping Functions as Fallback

In [39]:
def fetch_case_html(cnr=None, case_type=None, case_no=None, case_year=None):
    params = {}
    if cnr:
        params = {'p': 'cnrsearch', 'cnr': cnr}
    elif case_type and case_no and case_year:
        params = {'p': 'casestatus', 'case_type': case_type, 'case_no': case_no, 'case_year': case_year}
    else:
        return None
    with requests.Session() as session:
        res = session.get(BASE_URL, params=params)
        if res.status_code == 200:
            return res.text
    return None

def parse_case_listing(html, target_date_str):
    soup = BeautifulSoup(html, 'html.parser')
    for row in soup.find_all('tr'):
        if target_date_str in row.text:
            serial_no = row.find('td', class_='serialNo').text.strip() if row.find('td', class_='serialNo') else "N/A"
            court_name = row.find('td', class_='courtName').text.strip() if row.find('td', class_='courtName') else "N/A"
            return {'serial_no': serial_no, 'court_name': court_name}
    return None

def download_pdf(cnr):
    output_dir = '/content/pdf_downloads'
    os.makedirs(output_dir, exist_ok=True)
    url = f"{BASE_URL}?p=downloadPDF&cnr={cnr}"
    resp = requests.get(url)
    if 'pdf' in resp.headers.get('Content-Type', ''):
        path = os.path.join(output_dir, f"{cnr}.pdf")
        with open(path, 'wb') as f:
            f.write(resp.content)
        print(f"Downloaded PDF to {path}")
        return path
    print("No PDF available.")
    return None

def download_cause_list(date_obj):
    output_dir = '/content/cause_lists'
    os.makedirs(output_dir, exist_ok=True)
    date_str = date_obj.strftime('%d-%m-%Y')
    url = f"{BASE_URL}?p=causelist&date={date_str}"
    resp = requests.get(url)
    if resp.status_code == 200:
        path = os.path.join(output_dir, f"cause_list_{date_str}.html")
        with open(path, 'w', encoding='utf-8') as f:
            f.write(resp.text)
        print(f"Downloaded cause list to {path}")
        return path
    print("Failed to download cause list.")
    return None


Main Execution Flow

In [40]:
def main():
    (cnr, case_type, case_no, case_year, check_today, check_tomorrow,
     download_pdf_flag, download_cause_list_flag, api_key) = get_user_input()

    case_data = None
    if api_key and cnr:
        case_data = fetch_case_details_api(cnr, api_key)
        if case_data:
            print("Case details from API:")
            print(json.dumps(case_data, indent=4))

    if not case_data:
        target_date = None
        if check_today:
            target_date = datetime.today()
        elif check_tomorrow:
            target_date = datetime.today() + timedelta(days=1)

        html = fetch_case_html(cnr, case_type, case_no, case_year)
        if html and target_date:
            listing = parse_case_listing(html, target_date.strftime("%d-%m-%Y"))
            if listing:
                print(f"Case listed on {target_date.strftime('%d-%m-%Y')}:")
                print(listing)
            else:
                print(f"No listing found for {target_date.strftime('%d-%m-%Y')}.")

    if download_pdf_flag and cnr:
        download_pdf(cnr)

    if download_cause_list_flag:
        download_cause_list(datetime.today())

# Run the main function in the notebook
main()


Enter CNR number (leave blank if unknown): 
Enter Case Type (if no CNR): 
Enter Case Number (if no CNR): 
Enter Case Year (if no CNR): 
Check listings for today? (y/n): 
Check listings for tomorrow? (y/n): 
Download PDF if available? (y/n): 
Download full cause list for today? (y/n): 
Enter your E-Courts API key (leave blank to skip API): 


Install ipywidgets

In [41]:
!pip install ipywidgets
print("Step 1: ipywidgets installed")


Step 1: ipywidgets installed


 Import libraries and define scraper functions

In [42]:
import ipywidgets as widgets
from IPython.display import display, clear_output
import requests
from bs4 import BeautifulSoup
from datetime import datetime, timedelta

BASE_URL = "https://services.ecourts.gov.in/ecourtindia_v6/"

def fetch_case_html(cnr=None, case_type=None, case_no=None, case_year=None):
    params = {}
    if cnr:
        params = {'p': 'cnrsearch', 'cnr': cnr}
    elif case_type and case_no and case_year:
        params = {
            'p': 'casestatus',
            'case_type': case_type,
            'case_no': case_no,
            'case_year': case_year
        }
    else:
        return None
    response = requests.get(BASE_URL, params=params)
    if response.status_code == 200:
        return response.text
    return None

def parse_case_listing(html, date_str):
    soup = BeautifulSoup(html, 'html.parser')
    for row in soup.find_all('tr'):
        if date_str in row.text:
            serial = row.find('td', class_='serialNo')
            court = row.find('td', class_='courtName')
            return {
                'serial_no': serial.text.strip() if serial else 'N/A',
                'court_name': court.text.strip() if court else 'N/A',
            }
    return None


Create widgets with layout and validation

In [43]:
style = {'description_width': '150px'}
layout = widgets.Layout(width='350px')

cnr_input = widgets.Text(description="CNR Number:", style=style, layout=layout)
case_type_input = widgets.Text(description="Case Type:", style=style, layout=layout)
case_no_input = widgets.Text(description="Case Number:", style=style, layout=layout)
case_year_input = widgets.Text(description="Case Year:", style=style, layout=layout)

date_radio = widgets.RadioButtons(
    options=['Today', 'Tomorrow'], description='Listing Date:', style=style
)

scrape_button = widgets.Button(description="Scrape Case", button_style='primary', layout=widgets.Layout(width='150px'))

output_area = widgets.Output(layout={'border': '1px solid black'})


Define event handler with validation and rich display

In [44]:
def on_scrape_click(b):
    with output_area:
        clear_output()
        cnr = cnr_input.value.strip() or None
        case_type = case_type_input.value.strip() or None
        case_no = case_no_input.value.strip() or None
        case_year = case_year_input.value.strip() or None
        date_selected = date_radio.value

        if not cnr and not (case_type and case_no and case_year):
            print("Error: Provide either CNR or Case Type, Number and Year.")
            return
        print(f"Fetching case listing for {date_selected}...")

        target_date = datetime.today() if date_selected == 'Today' else datetime.today() + timedelta(days=1)
        html = fetch_case_html(cnr, case_type, case_no, case_year)
        if not html:
            print("Error: Failed to fetch data with provided input.")
            return

        listing = parse_case_listing(html, target_date.strftime('%d-%m-%Y'))
        if listing:
            print(f"Serial Number: {listing['serial_no']}")
            print(f"Court Name: {listing['court_name']}")
        else:
            print(f"No listing found for {date_selected}")

scrape_button.on_click(on_scrape_click)


Arrange widgets with containers and display interface

In [45]:
input_box = widgets.VBox([
    cnr_input,
    widgets.HBox([case_type_input, case_no_input, case_year_input]),
    date_radio,
    scrape_button
])

display(input_box, output_area)


Output(layout=Layout(border='1px solid black'))